# Skyfield
Biblioteka do obliczeń astronomicznych


## Przykładowe zastosowania
- śledzenie pozycji planet 
- określanie wschodów i zachodów słońca dla danej lokalizacji
- oblicznie momentów zaćmień i faz księżyca
- obsługa plików TLE i analiza pozycji satelitów

### Przykładowy kod 1 - odległości między planetami

In [30]:
from skyfield.api import load

# Getting current time
ts = load.timescale()
t = ts.now()

# This file covers planets positions 1900-2050).
planets = load('de421.bsp')
earth, venus = planets['earth'], planets['jupiter barycenter']

# Getting the position of Venus, viewed from Earth
astrometric = earth.at(t).observe(venus)
ra, dec, distance = astrometric.radec()

print('Earth to Venus\n')
print(f'Right Ascension: {ra}')                             # in hours where 1h = 15° and 0h is March equinox
print(f'Declination: {dec}')                                # in degrees where 0° is the celestial equator
print(f'Distance in astronomical units: {distance}')        # in AU where 1 AU is the average distance from Earth to the Sun
print(f"Distance in kilometers: {distance.km:,.0f} km")     # in km where 1 AU about 149mln km

Earth to Venus

Right Ascension: 04h 47m 09.93s
Declination: +22deg 01' 14.1"
Distance in astronomical units: 5.1831 au
Distance in kilometers: 775,380,990 km


### Przykładowy kod 2 - wschody i zachody słońca

In [27]:
from skyfield.api import load, wgs84
from skyfield import almanac  
import pytz

ts = load.timescale()
planets = load('de421.bsp')
earth, sun = planets['earth'], planets['sun']

# Wrocław coordinates and timezone
observer = wgs84.latlon(51.107883, 17.038538)
timezone = pytz.timezone('Europe/Warsaw')

t0 = ts.now()
t1 = ts.now() + 1

# Compute the times of sunrise and sunset
f = almanac.risings_and_settings(planets, sun, observer)
t_events, event_types = almanac.find_discrete(t0, t1, f)

# Event types: 1 = sunrise, 0 = sunset
sunrise = t_events[event_types == 1][0]
sunset = t_events[event_types == 0][0]

# Convert UTC to local time
sunset_local = sunset.utc_datetime().replace(tzinfo=pytz.utc).astimezone(timezone)
sunrise_local = sunrise.utc_datetime().replace(tzinfo=pytz.utc).astimezone(timezone)

print(f"Next sunset: {sunset_local:%Y-%m-%d %H:%M:%S %Z}")
print(f"Next sunrise: {sunrise_local:%Y-%m-%d %H:%M:%S %Z}")


Next sunset: 2025-03-14 17:53:21 CET
Next sunrise: 2025-03-14 06:09:30 CET


### Przykładowy kod 3 - fazy księżyca

In [28]:
from skyfield.api import load
from skyfield import almanac  

ts = load.timescale()
ephemeris_objects = load('de421.bsp')

t0 = ts.utc(2025, 3, 1)
t1 = ts.utc(2025, 4, 1)

# Create function for moon phases
f = almanac.moon_phases(ephemeris_objects)

# Find moon phases between t0 and t1
times, phases = almanac.find_discrete(t0, t1, f)

# Print results
for t, phase in zip(times, phases):
    print(f"{t.utc_datetime():%Y-%m-%d %H:%M:%S %Z} - {almanac.MOON_PHASES[phase]}")


2025-03-06 16:31:37 UTC - First Quarter
2025-03-14 06:54:39 UTC - Full Moon
2025-03-22 11:29:26 UTC - Last Quarter
2025-03-29 10:57:49 UTC - New Moon
